# 사전 작업

## 모듈 로드

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import gc
from tqdm import tqdm_notebook

In [2]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold, StratifiedKFold

In [3]:
from sklearn.preprocessing import LabelEncoder

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
pd.set_option('display.max_columns', 400)

# 통합 데이터

In [6]:
def drop_features(data, keyword, debug=False):
    if debug:
        data = data[:1000]
        
    logits = []
    for col in data.columns:
        if keyword in col:
            logits.append(col)
        else:
            pass
    
    return logits

## 데이터 로드

In [7]:
path = './data/'

In [8]:
# train = pd.read_csv(path + 'train_v4.csv')
# test = pd.read_csv(path + 'test_v4.csv')

In [9]:
# train.new_card_id_size.fillna(0, inplace=True)
# test.new_card_id_size.fillna(0, inplace=True)

In [10]:
raw_history = pd.read_csv(path + 'historical_transactions.csv')

In [11]:
history = raw_history.copy()

## Feature Engineering History

In [12]:
def null_cnt(x):
    return np.sum(x.isna() * 1)

In [13]:
def null_rate(x):
    return np.sum(x.isna() * 1) / len(x)

In [14]:
from scipy import stats
def mode(x):
    return stats.mode(x)[0][0]

In [15]:
def most_value_cnt(x):
    return x.value_counts().values[0]

In [16]:
history.head(3)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37


### 거래가 승인 되고, 오프라인 거래의 city_id

In [17]:
temp = history[history.authorized_flag == 'Y']
temp = temp[temp.category_1 == 'N']

In [18]:
temp = temp.groupby('card_id').agg({'city_id': [mode, 'nunique']})
temp.head(1)

city_id        
                   mode nunique
card_id                        
C_ID_00007093c1     244       3

In [19]:
temp.columns = ['hist_city_id_mode_authorized_flag_category_1_Y', 'hist_city_id_nunique_authorized_flag_category_1_Y']
temp.reset_index(inplace=True)
temp.head(1)

,card_id,hist_city_id_mode_authorized_flag_category_1_Y,hist_city_id_nunique_authorized_flag_category_1_Y
0,C_ID_00007093c1,244,3


In [20]:
train = train.merge(temp, on='card_id', how='left')
test = test.merge(temp, on='card_id', how='left')

In [21]:
del temp
gc.collect()

28

### 거래가 승인된 category_1

In [22]:
temp = history[history.authorized_flag == 'Y']

In [23]:
temp.category_1 = temp.category_1.map({'Y':1, 'N':0})

In [24]:
temp = temp.groupby('card_id').agg({'category_1':['mean', 'sum', 'size']})
temp.head(1)

category_1         
                      mean sum size
card_id                            
C_ID_00007093c1   0.210526  24  114

In [25]:
temp.columns = ['hist_category_1_authorized_flag_Y_mean', 'hist_category_1_authorized_flag_Y_sum', 'hist_category_1_authorized_flag_Y_size']
temp.reset_index(inplace=True)
temp.head(1)

,card_id,hist_category_1_authorized_flag_Y_mean,hist_category_1_authorized_flag_Y_sum,hist_category_1_authorized_flag_Y_size
0,C_ID_00007093c1,0.210526,24,114


In [26]:
temp.hist_category_1_authorized_flag_Y_mean = np.round(temp.hist_category_1_authorized_flag_Y_mean, 4)

In [27]:
train = train.merge(temp, on='card_id', how='left')
test = test.merge(temp, on='card_id', how='left')

In [28]:
del temp
gc.collect()

35

### 거래가 승인된 installments

In [17]:
def has_999(data):
    # 999가 없을 때
    if data[data == 999].__len__() == 0:
        return 0
    # 999가 있을 때
    else:
        return 1

In [18]:
def cnt_std(data):
    value = data.value_counts().std()
    if np.isnan(value):
        return 0
    else:
        return value

In [31]:
history.installments.replace(-1, np.nan, inplace=True)

In [32]:
temp = history[history.authorized_flag == 'Y']

In [33]:
temp = temp.groupby('card_id').agg({'installments':[mode, 'size', 'mean', 'max', 'var', 'min', null_cnt, null_rate, cnt_std]})
temp.head(1)

installments                                              \
                        mode size      mean  max       var  min null_cnt   
card_id                                                                    
C_ID_00007093c1          1.0  114  1.289474  6.0  0.632278  1.0      0.0   

                                      
                null_rate    cnt_std  
card_id                               
C_ID_00007093c1       0.0  41.069453

In [34]:
temp.columns = ['hist_installments_authorized_flag_Y_mode', 'hist_installments_authorized_flag_Y_size', 'hist_installments_authorized_flag_Y_mean', 'hist_installments_authorized_flag_Y_max', 'hist_installments_authorized_flag_Y_var', 'hist_installments_authorized_flag_Y_min', 'hist_installments_authorized_flag_Y_null_cnt', 'hist_installments_authorized_flag_Y_null_rate', 'hist_installments_authorized_flag_Y_cnt_std']
temp.reset_index(inplace=True)
temp.head(1)

,card_id,hist_installments_authorized_flag_Y_mode,hist_installments_authorized_flag_Y_size,hist_installments_authorized_flag_Y_mean,hist_installments_authorized_flag_Y_max,hist_installments_authorized_flag_Y_var,hist_installments_authorized_flag_Y_min,hist_installments_authorized_flag_Y_null_cnt,hist_installments_authorized_flag_Y_null_rate,hist_installments_authorized_flag_Y_cnt_std
0,C_ID_00007093c1,1.0,114,1.289474,6.0,0.632278,1.0,0.0,0.0,41.069453


In [35]:
train = train.merge(temp, on='card_id', how='left')
test = test.merge(temp, on='card_id', how='left')

In [36]:
del temp
gc.collect()

42

### category_3

In [37]:
history.category_3 = history.category_3.fillna('D')

### 승인된 거래 중 merchant_category_id

In [38]:
temp = history[history.authorized_flag == 'Y']

In [39]:
temp = temp.groupby('card_id').agg({'merchant_category_id':[mode, 'nunique', null_cnt, cnt_std]})

In [40]:
temp.columns = ['hist_merchant_category_id_authorized_flag_Y_mode', 'hist_merchant_category_id_authorized_flag_Y_nunique', 'hist_merchant_category_id_authorized_flag_Y_null_cnt', 'hist_merchant_category_id_authorized_flag_Y_cnt_std']
temp.reset_index(inplace=True)
temp.head(1)

,card_id,hist_merchant_category_id_authorized_flag_Y_mode,hist_merchant_category_id_authorized_flag_Y_nunique,hist_merchant_category_id_authorized_flag_Y_null_cnt,hist_merchant_category_id_authorized_flag_Y_cnt_std
0,C_ID_00007093c1,307,18,0,10.278476


In [41]:
temp.hist_merchant_category_id_authorized_flag_Y_cnt_std = np.round(temp.hist_merchant_category_id_authorized_flag_Y_cnt_std, 3)

In [42]:
train = train.merge(temp, on='card_id', how='left')
test = test.merge(temp, on='card_id', how='left')

In [43]:
del temp
gc.collect()

28

### merchant_id

In [44]:
le = LabelEncoder()
le.fit(history.merchant_id.fillna('NULL').values)
history.merchant_id = le.transform(history.merchant_id.fillna('NULL'))
history.loc[history.merchant_id == 326311, 'merchant_id'] = np.nan

In [45]:
temp = history[history.authorized_flag == 'Y']

In [46]:
temp = temp.groupby('card_id').agg({'merchant_id': [cnt_std]})
temp.head(1)

,merchant_id
,cnt_std
card_id,
C_ID_00007093c1,6.943841


In [47]:
temp.columns = ['hist_merchant_id_authorized_flag_Y_cnt_std']
temp.reset_index(inplace=True)
temp.head(1)

,card_id,hist_merchant_id_authorized_flag_Y_cnt_std
0,C_ID_00007093c1,6.943841


In [48]:
temp.hist_merchant_id_authorized_flag_Y_cnt_std = np.round(temp.hist_merchant_id_authorized_flag_Y_cnt_std, 3)

In [49]:
train = train.merge(temp, on='card_id', how='left')
test = test.merge(temp, on='card_id', how='left')

In [50]:
del temp
gc.collect()

42

In [51]:
temp = history.groupby('card_id').agg({'merchant_id': [mode, 'nunique', null_cnt, cnt_std]})
temp.head(1)

merchant_id                           
                       mode nunique null_cnt   cnt_std
card_id                                               
C_ID_00007093c1    188773.0      29      0.0  8.794171

In [52]:
temp.columns = ['hist_merchant_id_mode', 'hist_merchant_id_nunique2', 'hist_merchant_id_null_cnt', 'hist_merchant_id_cnt_std']
temp.reset_index(inplace=True)
temp.head(1)

,card_id,hist_merchant_id_mode,hist_merchant_id_nunique2,hist_merchant_id_null_cnt,hist_merchant_id_cnt_std
0,C_ID_00007093c1,188773.0,29,0.0,8.794171


In [53]:
temp.hist_merchant_id_cnt_std = np.round(temp.hist_merchant_id_cnt_std, 3)

In [54]:
train = train.merge(temp, on='card_id', how='left')
test = test.merge(temp, on='card_id', how='left')

In [55]:
del temp
gc.collect()

28

### month_lag

In [56]:
temp = history[history.authorized_flag == 'Y']
temp = temp[temp.category_1 == 'Y']
temp = history.groupby('card_id').agg({'month_lag': [mode, 'min', 'max', cnt_std]})
temp.head(1)

month_lag                  
                     mode min max   cnt_std
card_id                                    
C_ID_00007093c1        -8 -12   0  4.701336

In [57]:
temp.columns = ['hist_month_lag_authorized_flag_Y_category_1_Y_mode', 'hist_month_lag_authorized_flag_Y_category_1_Y_min', 'hist_month_lag_authorized_flag_Y_category_1_Y_max', 'hist_month_lag_authorized_flag_Y_category_1_Y_cnt_std']
temp.reset_index(inplace=True)
temp.head(1)

,card_id,hist_month_lag_authorized_flag_Y_category_1_Y_mode,hist_month_lag_authorized_flag_Y_category_1_Y_min,hist_month_lag_authorized_flag_Y_category_1_Y_max,hist_month_lag_authorized_flag_Y_category_1_Y_cnt_std
0,C_ID_00007093c1,-8,-12,0,4.701336


In [58]:
temp.hist_month_lag_authorized_flag_Y_category_1_Y_cnt_std = np.round(temp.hist_month_lag_authorized_flag_Y_category_1_Y_cnt_std, 3)

In [59]:
train = train.merge(temp, on='card_id', how='left')
test = test.merge(temp, on='card_id', how='left')

In [60]:
del temp
gc.collect()

28

In [61]:
temp = history[history.authorized_flag == 'Y']
temp = temp[temp.category_1 == 'N']
temp = history.groupby('card_id').agg({'month_lag': [mode, 'min', 'max', cnt_std]})
temp.head(1)

month_lag                  
                     mode min max   cnt_std
card_id                                    
C_ID_00007093c1        -8 -12   0  4.701336

In [62]:
temp.columns = ['hist_month_lag_authorized_flag_Y_category_1_N_mode', 'hist_month_lag_authorized_flag_Y_category_1_N_min', 'hist_month_lag_authorized_flag_Y_category_1_N_max', 'hist_month_lag_authorized_flag_Y_category_1_N_cnt_std']
temp.reset_index(inplace=True)
temp.head(1)

,card_id,hist_month_lag_authorized_flag_Y_category_1_N_mode,hist_month_lag_authorized_flag_Y_category_1_N_min,hist_month_lag_authorized_flag_Y_category_1_N_max,hist_month_lag_authorized_flag_Y_category_1_N_cnt_std
0,C_ID_00007093c1,-8,-12,0,4.701336


In [63]:
temp.hist_month_lag_authorized_flag_Y_category_1_N_cnt_std = np.round(temp.hist_month_lag_authorized_flag_Y_category_1_N_cnt_std, 3)

In [64]:
train = train.merge(temp, on='card_id', how='left')
test = test.merge(temp, on='card_id', how='left')

In [65]:
del temp
gc.collect()

28

### category_2

In [66]:
temp = history[history.authorized_flag == 'Y']

In [67]:
temp = temp.groupby('card_id').agg({'category_2': [mode, 'nunique', null_cnt, cnt_std]})

In [68]:
temp.head(1)

category_2                            
                      mode nunique null_cnt    cnt_std
card_id                                               
C_ID_00007093c1        3.0       2     24.0  62.225397

In [69]:
temp.columns = ['hist_category_2_authorized_flag_Y_mode', 'hist_category_2_authorized_flag_Y_nunique', 'hist_category_2_authorized_flag_Y_null_cnt', 'hist_category_2_authorized_flag_Y_cnt_std']
temp.reset_index(inplace=True)
temp.head(1)

,card_id,hist_category_2_authorized_flag_Y_mode,hist_category_2_authorized_flag_Y_nunique,hist_category_2_authorized_flag_Y_null_cnt,hist_category_2_authorized_flag_Y_cnt_std
0,C_ID_00007093c1,3.0,2,24.0,62.225397


In [70]:
train = train.merge(temp, on='card_id', how='left')
test = test.merge(temp, on='card_id', how='left')

In [71]:
del temp
gc.collect()

28

### subsector

In [72]:
# temp = history[history.authorized_flag == 'Y']
# temp = temp.groupby('card_id').agg({'subsector_id': [mode, 'nunique', null_cnt, cnt_std]})

In [73]:
# temp.columns = ['hist_subsector_id_authorized_flag_Y_mode', 'hist_subsector_id_authorized_flag_Y_nunique', 'hist_subsector_id_authorized_flag_Y_null_cnt', 'hist_subsector_id_authorized_flag_Y_cnt_std']
# temp.reset_index(inplace=True)
# temp.head(1)

In [74]:
# train = train.merge(temp, on='card_id', how='left')
# test = test.merge(temp, on='card_id', how='left')

In [75]:
# del temp
# gc.collect()

## logging

In [18]:
# train.to_csv('./data_feature_engineering/train_3-73218.csv', index=False)
# test.to_csv('./data_feature_engineering/test_3-73218.csv', index=False)
train = pd.read_csv('./data_feature_engineering/train_3-73218.csv')
test = pd.read_csv('./data_feature_engineering/test_3-73218.csv')

### purchase_date

In [33]:
for new_merchant_df in [history]:
    # purchase date
    new_merchant_df['purchase_year_month'] = new_merchant_df.purchase_date.str[:7]
    new_merchant_df['purchase_year_month_day'] = new_merchant_df.purchase_date.str[:10]
    new_merchant_df['purchase_date'] = pd.to_datetime(new_merchant_df['purchase_date'])
    new_merchant_df['purchase_year_month_day'] = pd.to_datetime(new_merchant_df['purchase_year_month_day'])
    print(1)
    new_merchant_df['purchase_year'] = new_merchant_df['purchase_date'].dt.year
    print(2)
    new_merchant_df['purchase_month'] = new_merchant_df['purchase_date'].dt.month
    print(3)
    new_merchant_df['purchase_day'] = new_merchant_df['purchase_date'].dt.day
    new_merchant_df['purchase_hour'] = new_merchant_df['purchase_date'].dt.hour
    new_merchant_df['purchase_dayofweek'] = new_merchant_df['purchase_date'].dt.dayofweek
    new_merchant_df['purchase_weekofyear'] = new_merchant_df['purchase_date'].dt.weekofyear
    new_merchant_df['purchase_weekend'] = (new_merchant_df['purchase_date'].dt.weekday >=5).astype(int)
    print(4)
    new_merchant_df['purchase_date_total_day'] = pd.to_timedelta(new_merchant_df['purchase_year_month_day']).dt.total_seconds() / (60 * 60 * 24)
    print(5)
    new_merchant_df['month_diff_from_last_trade'] = ((datetime.datetime(2018, 4, 30, 23, 59, 59) - new_merchant_df['purchase_date']).dt.days)//30
    new_merchant_df['month_diff_from_last_trade'] += new_merchant_df['month_lag']
    new_merchant_df['month_diff_from_first_trade'] = ((datetime.datetime(2017, 1, 1, 0, 0, 0) - new_merchant_df['purchase_date']).dt.days)//30
    new_merchant_df['month_diff_from_first_trade'] += new_merchant_df['month_lag']
    new_merchant_df['month_diff_from_today'] = ((datetime.datetime.today() - new_merchant_df['purchase_date']).dt.days)//30
    new_merchant_df['month_diff_from_today'] += new_merchant_df['month_lag']
    print(6)
    

1
2
3
4
5
6


#### purchase total day

In [40]:
temp = history.groupby('card_id').agg({'purchase_date_total_day': [mode, 'sum', 'mean', 'std', 'max', 'min', cnt_std]})

In [41]:
temp.columns = ['hist_purchase_date_total_day_mode', 'hist_purchase_date_total_day_sum', 'hist_purchase_date_total_day_mean', 'hist_purchase_date_total_day_std', 'hist_purchase_date_total_day_max', 'hist_purchase_date_total_day_min' ,'hist_purchase_date_total_day_cnt_std']
temp.reset_index(inplace=True)
temp.head(1)

,card_id,hist_purchase_date_total_day_mode,hist_purchase_date_total_day_sum,hist_purchase_date_total_day_mean,hist_purchase_date_total_day_std,hist_purchase_date_total_day_max,hist_purchase_date_total_day_min,hist_purchase_date_total_day_cnt_std
0,C_ID_00007093c1,17273.0,2591980.0,17395.838926,104.216256,17589.0,17211.0,1.288961


In [42]:
train = train.merge(temp, on='card_id', how='left')
test = test.merge(temp, on='card_id', how='left')

del temp
gc.collect()

275

#### purchase year month

In [86]:
temp = history.groupby('card_id').agg({'purchase_year_month': [mode, cnt_std]})

In [87]:
temp.columns = ['hist_purchase_year_month_mode', 'hist_purchase_year_month_cnt_std']
temp.reset_index(inplace=True)
temp.head(1)

,card_id,hist_purchase_year_month_mode,hist_purchase_year_month_cnt_std
0,C_ID_00007093c1,2017-06,4.701336


In [88]:
train = train.merge(temp, on='card_id', how='left')
test = test.merge(temp, on='card_id', how='left')

del temp
gc.collect()

167

#### month diff

In [110]:
temp = history.groupby('card_id').agg({'month_diff_from_last_trade': [mode, 'sum', 'max', 'min', 'std', 'var', cnt_std], 'month_diff_from_first_trade': [mode, 'sum', 'max', 'min', 'std', 'var', cnt_std], 'month_diff_from_today':[mode, 'sum', 'max', 'min', 'std', 'var', cnt_std]})

In [113]:
temp

month_diff_from_last_trade                                    \
                                      mode   sum max min       std       var   
card_id                                                                        
C_ID_00007093c1                          2   318   3   2  0.342047  0.116996   
C_ID_0001238066                          2   247   3   2  0.090167  0.008130   
C_ID_0001506ef0                          2   139   3   2  0.310275  0.096270   
C_ID_0001793786                          6  1322   7   6  0.326150  0.106374   
C_ID_000183fdda                          2   288   3   1  0.236525  0.055944   
C_ID_00024e244b                          2   158   3   2  0.440215  0.193789   
C_ID_0002709b5a                          2   149   3   2  0.199886  0.039954   
C_ID_00027503e2                          3   126   3   3  0.000000  0.000000   
C_ID_000298032a                          3    90   3   3  0.000000  0.000000   
C_ID_0002ba3c2e                          6   428   7   6  0.320455  0.102692   
C_ID_0002c7c2c1                          3   204   4   3  0.208373  0.043419   
C_ID_00032df08f                          5   740   6   5  0.305599  0.093391   
C_ID_0003754056                          3   109   4   3  0.166667  0.027778   
C_ID_000377f6a0                          2    56   2   2  0.000000  0.000000   
C_ID_0003be3c83                          2   347   3   2  0.168973  0.028552   
C_ID_0003f41435                          2   542   3   2  0.239486  0.057353   
C_ID_00042d509c                          4   257   5   4  0.355139  0.126124   
C_ID_0004587331                          3    42   3   3  0.000000  0.000000   
C_ID_0004725b87                          2   163   3   2  0.287664  0.082751   
C_ID_0004888ddd                          3   126   4   2  0.345705  0.119512   
C_ID_0004b68c49                          4    21   5   4  0.447214  0.200000   
C_ID_0004c2a5ab                          2   156   3   2  0.346212  0.119863   
C_ID_00057b99fe                          2    76   3   2  0.382385  0.146218   
C_ID_000599daf9                          2    71   3   2  0.169031  0.028571   
C_ID_0005b2f279                          4    65   5   4  0.250000  0.062500   
C_ID_0005b5804f                          2    16   2   2  0.000000  0.000000   
C_ID_0005f16cc8                          2   320   3   1  0.331652  0.109993   
C_ID_0006152db8                          2   184   3   2  0.348536  0.121477   
C_ID_000616f4a8                          2    77   3   2  0.162221  0.026316   
C_ID_000664aa02                          2    70   2   2  0.000000  0.000000   
...                                    ...   ...  ..  ..       ...       ...   
C_ID_fffb79fb56                         11   233  12  11  0.300793  0.090476   
C_ID_fffb9ea3f6                          2   117   3   2  0.131306  0.017241   
C_ID_fffba72dc5                          2   163   3   2  0.111111  0.012346   
C_ID_fffbdf036b                         11  1080  12  11  0.342444  0.117268   
C_ID_fffbee5c24                          3    69   4   3  0.351250  0.123377   
C_ID_fffc96bf24                          2   462   3   2  0.336011  0.112903   
C_ID_fffcb74f49                          2   223   3   2  0.210173  0.044173   
C_ID_fffcf66e12                          2    72   3   2  0.327035  0.106952   
C_ID_fffd070e0e                          2    83   2   1  0.154303  0.023810   
C_ID_fffd1207a8                          2  1668   3   1  0.291628  0.085047   
C_ID_fffd24dcf3                          2    28   2   2  0.000000  0.000000   
C_ID_fffd337484                          8   275   9   8  0.287902  0.082888   
C_ID_fffd345048                          2   139   3   2  0.120386  0.014493   
C_ID_fffd351e7f                          2   477   3   1  0.295075  0.087069   
C_ID_fffd93c8bf                          2   268   3   1  0.122628  0.015038   
C_ID_fffd943b91                          3   194   4   3  0.175368  0.030754   
C_ID_fffdad8aa0 

In [117]:
col = ['month_diff_from_last_trade', 'month_diff_from_first_trade', 'month_diff_from_today']
final_col = []
for c in col:
    final_col += ['hist_' + c + '_' + v for v in ['mode', 'sum', 'max', 'min', 'std', 'var', 'cnt_std']]

final_col
temp.columns = final_col
temp.reset_index(inplace=True)
temp.head(1)

,card_id,hist_month_diff_from_last_trade_mode,hist_month_diff_from_last_trade_sum,hist_month_diff_from_last_trade_max,hist_month_diff_from_last_trade_min,hist_month_diff_from_last_trade_std,hist_month_diff_from_last_trade_var,hist_month_diff_from_last_trade_cnt_std,hist_month_diff_from_first_trade_mode,hist_month_diff_from_first_trade_sum,hist_month_diff_from_first_trade_max,hist_month_diff_from_first_trade_min,hist_month_diff_from_first_trade_std,hist_month_diff_from_first_trade_var,hist_month_diff_from_first_trade_cnt_std,hist_month_diff_from_today_mode,hist_month_diff_from_today_sum,hist_month_diff_from_today_max,hist_month_diff_from_today_min,hist_month_diff_from_today_std,hist_month_diff_from_today_var,hist_month_diff_from_today_cnt_std
0,C_ID_00007093c1,2,318,3,2,0.342047,0.116996,77.074639,-14,-2099,-14,-15,0.28315,0.080174,86.974134,12,1776,12,11,0.27304,0.074551,88.388348


In [118]:
train = train.merge(temp, on='card_id', how='left')
test = test.merge(temp, on='card_id', how='left')

del temp
gc.collect()

91

In [153]:
temp = history.groupby('card_id').agg({'month_diff_from_last_trade': ['skew']})

In [155]:
temp.columns = ['hist_month_diff_from_last_trade_skew']
temp.reset_index(inplace=True)
temp.head(1)

,card_id,hist_month_diff_from_last_trade_skew
0,C_ID_00007093c1,2.16782


In [156]:
train = train.merge(temp, on='card_id', how='left')
test = test.merge(temp, on='card_id', how='left')

del temp
gc.collect()

20

#### purchase_date

In [222]:
def duration(data):
    return (data.max() - data.min()).days

In [223]:
temp = history.groupby(['card_id']).agg({'purchase_year_month_day': ['max', 'min', duration]})

In [224]:
temp.columns = ['hist_purchase_year_month_day_max', 'hist_purchase_year_month_day_min', 'hist_purchase_year_month_day_duration']
temp.reset_index(inplace=True)
temp.head(1)

,card_id,hist_purchase_year_month_day_max,hist_purchase_year_month_day_min,hist_purchase_year_month_day_duration
0,C_ID_00007093c1,2018-02-27,2017-02-14,378


In [225]:
train = train.merge(temp, on='card_id', how='left')
test = test.merge(temp, on='card_id', how='left')

del temp
gc.collect()

7

### purchase_amount

In [19]:
# history['purchase_amount_new'] = np.round(history['purchase_amount'] / 0.00150265118 + 497.06,2)

In [20]:
# temp = history[history.authorized_flag == 'Y']
# temp = temp.groupby(['card_id']).agg({'purchase_amount_new':[mode, 'sum', 'mean', 'var', 'max', 'min', 'skew', 'std']})

In [21]:
# temp.columns = ['hist_purchase_amount_authorized_flag_Y_new_mode', 'hist_purchase_amount_authorized_flag_Y_new_sum', 'hist_purchase_amount_authorized_flag_Y_new_mean', 'hist_purchase_amount_authorized_flag_Y_new_var', 'hist_purchase_amount_authorized_flag_Y_new_max', 'hist_purchase_amount_authorized_flag_Y_new_min', 'hist_purchase_amount_authorized_flag_Y_new_skew', 'hist_purchase_amount_authorized_flag_Y_new_std']
# temp.reset_index(inplace=True)
# temp.head(1)

In [22]:
# train = train.merge(temp, on='card_id', how='left')
# test = test.merge(temp, on='card_id', how='left')

In [23]:
# del temp
# gc.collect()

In [24]:
# temp = history[history.authorized_flag == 'N']
# temp = temp.groupby(['card_id']).agg({'purchase_amount_new':[mode, 'sum', 'mean', 'var', 'max', 'min', 'skew', 'std']})

In [25]:
# temp.columns = ['hist_purchase_amount_authorized_flag_N_new_mode', 'hist_purchase_amount_authorized_flag_N_new_sum', 'hist_purchase_amount_authorized_flag_N_new_mean', 'hist_purchase_amount_authorized_flag_N_new_var', 'hist_purchase_amount_authorized_flag_N_new_max', 'hist_purchase_amount_authorized_flag_N_new_min', 'hist_purchase_amount_authorized_flag_N_new_skew', 'hist_purchase_amount_authorized_flag_N_new_std']
# temp.reset_index(inplace=True)
# temp.head(1)

In [26]:
# train = train.merge(temp, on='card_id', how='left')
# test = test.merge(temp, on='card_id', how='left')

In [27]:
# del temp
# gc.collect()

In [28]:
# history.purchase_amount_new.describe()

## logging

In [29]:
train = pd.read_csv('./data_feature_engineering/train_3-70479.csv')
test = pd.read_csv('./data_feature_engineering/test_3-70479.csv')

## Feature Engineering New History

In [114]:
new_history = pd.read_csv('./data/new_merchant_transactions.csv')

### 거래가 승인 된 city_id

In [149]:
temp = new_history[new_history.authorized_flag == 'Y']
# temp = temp[temp.category_1 == 'N']

In [150]:
temp = temp.groupby('card_id').agg({'city_id': [mode, 'nunique']})
temp.head(1)

city_id        
                   mode nunique
card_id                        
C_ID_00007093c1      69       2

In [151]:
temp.columns = ['new_city_id_mode_authorized_flag_Y', 'new_city_id_nunique_authorized_flag_Y']
temp.reset_index(inplace=True)
temp.head(1)

,card_id,new_city_id_mode_authorized_flag_Y,new_city_id_nunique_authorized_flag_Y
0,C_ID_00007093c1,69,2


In [152]:
train = train.merge(temp, on='card_id', how='left')
test = test.merge(temp, on='card_id', how='left')

In [153]:
del temp
gc.collect()

14

### 거래가 승인된 category_1

In [171]:
temp = new_history[new_history.authorized_flag == 'Y']

In [172]:
temp.category_1 = temp.category_1.map({'Y':1, 'N':0})

In [173]:
temp = temp.groupby('card_id').agg({'category_1':['mean', 'sum', 'size']})
temp.head(1)

category_1         
                      mean sum size
card_id                            
C_ID_00007093c1        0.0   0    2

In [174]:
temp.columns = ['new_category_1_authorized_flag_Y_mean', 'new_category_1_authorized_flag_Y_sum', 'new_category_1_authorized_flag_Y_size']
temp.reset_index(inplace=True)
temp.head(1)

,card_id,new_category_1_authorized_flag_Y_mean,new_category_1_authorized_flag_Y_sum,new_category_1_authorized_flag_Y_size
0,C_ID_00007093c1,0.0,0,2


In [176]:
temp.new_category_1_authorized_flag_Y_mean = np.round(temp.new_category_1_authorized_flag_Y_mean, 4)

In [177]:
train = train.merge(temp, on='card_id', how='left')
test = test.merge(temp, on='card_id', how='left')

In [178]:
del temp
gc.collect()

104

### 거래가 승인된 installments

In [201]:
def has_999(data):
    # 999가 없을 때
    if data[data == 999].__len__() == 0:
        return 0
    # 999가 있을 때
    else:
        return 1

In [202]:
def cnt_std(data):
    value = data.value_counts().std()
    if np.isnan(value):
        return 0
    else:
        return value

In [203]:
new_history.installments.replace(-1, np.nan, inplace=True)

In [204]:
temp = new_history[new_history.authorized_flag == 'Y']

In [205]:
temp = temp.groupby('card_id').agg({'installments':[mode, 'size', 'mean', 'max', 'var', 'min', null_cnt, null_rate, cnt_std]})
temp.head(1)

installments                                              \
                        mode size mean  max  var  min null_cnt null_rate   
card_id                                                                    
C_ID_00007093c1          1.0    2  1.0  1.0  0.0  1.0      0.0       0.0   

                         
                cnt_std  
card_id                  
C_ID_00007093c1     0.0

In [206]:
temp.columns = ['new_installments_authorized_flag_Y_mode', 'new_installments_authorized_flag_Y_size', 'new_installments_authorized_flag_Y_mean', 'new_installments_authorized_flag_Y_max', 'new_installments_authorized_flag_Y_var', 'new_installments_authorized_flag_Y_min', 'new_installments_authorized_flag_Y_null_cnt', 'new_installments_authorized_flag_Y_null_rate', 'new_installments_authorized_flag_Y_cnt_std']
temp.reset_index(inplace=True)
temp.head(1)

,card_id,new_installments_authorized_flag_Y_mode,new_installments_authorized_flag_Y_size,new_installments_authorized_flag_Y_mean,new_installments_authorized_flag_Y_max,new_installments_authorized_flag_Y_var,new_installments_authorized_flag_Y_min,new_installments_authorized_flag_Y_null_cnt,new_installments_authorized_flag_Y_null_rate,new_installments_authorized_flag_Y_cnt_std
0,C_ID_00007093c1,1.0,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0


In [207]:
train = train.merge(temp, on='card_id', how='left')
test = test.merge(temp, on='card_id', how='left')

In [208]:
del temp
gc.collect()

57

## history + new_history

In [305]:
result = history.append(new_history)

In [306]:
result['purchase_date_year_month'] = result.purchase_date.str[:7]

In [307]:
temp = result.groupby(['card_id', 'purchase_date_year_month']).size()

In [308]:
date_list = ['2017-01','2017-02','2017-03','2017-04','2017-05','2017-06','2017-07','2017-08','2017-09','2017-10','2017-11','2017-12','2018-01','2018-02','2018-03','2018-04']

In [310]:
meta = result.groupby('card_id').size().to_frame().reset_index().drop([0], axis=1)

In [312]:
for date in date_list:
    try:
        meta = meta.merge(temp.loc[:, date].to_frame().reset_index().rename(columns={0:date}), on='card_id', how='left')
        print(date)
    except:
        meta[date] = 0
        print(date)

2017-01
2017-02
2017-03
2017-04
2017-05
2017-06
2017-07
2017-08
2017-09
2017-10
2017-11
2017-12
2018-01
2018-02
2018-03
2018-04


In [316]:
train = train.merge(meta, on='card_id', how='left')

In [317]:
test = test.merge(meta, on='card_id', how='left')

In [ ]:
result[:100].groupby(['card_id', 'purchase_date_year_month']).size()

In [441]:
hist2 = pd.read_csv('./data/newFE_0213.csv')

In [443]:
hist2.drop(list(set(train.columns).intersection(set(hist2.columns))),axis=1, inplace=True)

In [444]:
hist2.head()

,new_card_id,new_fromRefDate_sum,new_fromRefDate_min,new_fromRefDate_max,new_fromRefDate_mean,new_fromRefDate_median,new_fromRefDate_var,new_fromRefDate_quantileRange,new_month_lag_median,new_month_lag_quantileRange,new_purchase_date_total_day_max,new_purchase_date_total_day_min,new_purchase_date_total_day_mean,new_purchase_date_total_day_var,new_purchase_date_total_day_skew,new_month_diff_from_trade_max,new_month_diff_from_trade_min,new_month_diff_from_trade_mean,new_month_diff_from_trade_var,new_month_diff_from_trade_skew,new_month_diff_from_today_max,new_month_diff_from_today_min,new_month_diff_from_today_mean,new_month_diff_from_today_var,new_month_diff_from_today_skew,new_city_id_count,new_merchant_id_mode,new_merchant_visit_sum,new_merchant_visit_mean,new_merchant_visit_min,new_merchant_visit_max,new_merchant_visit_nunique,new_merchant_visit_size,new_merchant_visit_mode,new_merchant_try_mean,new_merchant_try_std,new_merchant_try_min,new_merchant_try_max,new_merchant_try_nunique,new_merchant_try_size,new_merchant_try_mode,new_installments_median,new_installments_quantileRange,new_purchase_amount_median,new_purchase_amount_quantileRange,new_category_1_nunique,new_category_1_var,new_category_3_nunique,new_category_3_var,new_category_2_var,new_category_2_nunique,new_category_2_min_mean,new_category_2_max_mean,new_category_2_sum_mean,new_category_3_min_mean,new_category_3_max_mean,new_category_3_sum_mean,new_merchant_group_id_mode,new_merchant_group_id_nunique,new_numerical_1_sum,new_numerical_1_max,new_numerical_1_min,new_numerical_1_mean,new_numerical_1_median,new_numerical_1_var,new_numerical_1_quantileRange,new_numerical_2_sum,new_numerical_2_max,new_numerical_2_min,new_numerical_2_mean,new_numerical_2_median,new_numerical_2_var,new_numerical_2_quantileRange,new_most_recent_sales_range_mode,new_most_recent_sales_range_nunique,new_most_recent_sales_range_mean,new_most_recent_sales_range_var,new_most_recent_purchases_range_mode,new_most_recent_purchases_range_nunique,new_most_recent_purchases_range_mean,new_most_recent_purchases_range_var,new_category_4_mode,new_category_4_nunique,new_category_4_mean,new_category_4_var,new_category_5_mode,new_category_5_nunique,new_category_5_mean,new_category_5_var,new_avg_sales_lag3_sum,new_avg_sales_lag3_max,new_avg_sales_lag3_min,new_avg_sales_lag3_mean,new_avg_sales_lag3_median,new_avg_sales_lag3_var,new_avg_sales_lag3_quantileRange,new_avg_purchases_lag3_sum,new_avg_purchases_lag3_max,new_avg_purchases_lag3_min,new_avg_purchases_lag3_mean,new_avg_purchases_lag3_median,new_avg_purchases_lag3_var,new_avg_purchases_lag3_quantileRange,new_active_months_lag3_sum,new_active_months_lag3_max,new_active_months_lag3_min,new_active_months_lag3_mean,new_active_months_lag3_median,new_active_months_lag3_var,new_active_months_lag3_quantileRange,new_avg_sales_lag6_sum,new_avg_sales_lag6_max,new_avg_sales_lag6_min,new_avg_sales_lag6_mean,new_avg_sales_lag6_median,new_avg_sales_lag6_var,new_avg_sales_lag6_quantileRange,new_avg_purchases_lag6_sum,new_avg_purchases_lag6_max,new_avg_purchases_lag6_min,new_avg_purchases_lag6_mean,new_avg_purchases_lag6_median,new_avg_purchases_lag6_var,new_avg_purchases_lag6_quantileRange,new_active_months_lag6_sum,new_active_months_lag6_max,new_active_months_lag6_min,new_active_months_lag6_mean,new_active_months_lag6_median,new_active_months_lag6_var,new_active_months_lag6_quantileRange,new_avg_sales_lag12_sum,new_avg_sales_lag12_max,new_avg_sales_lag12_min,new_avg_sales_lag12_mean,new_avg_sales_lag12_median,new_avg_sales_lag12_var,new_avg_sales_lag12_quantileRange,new_avg_purchases_lag12_sum,new_avg_purchases_lag12_max,new_avg_purchases_lag12_min,new_avg_purchases_lag12_mean,new_avg_purchases_lag12_median,new_avg_purchases_lag12_var,new_avg_purchases_lag12_quantileRange,new_active_months_lag12_sum,new_active_months_lag12_max,new_active_months_lag12_min,new_active_months_lag12_mean,new_active_months_lag12_median,new_active_months_lag12_var,new_active_months_lag12_quantile

In [446]:
train = train.merge(hist2, left_on='card_id', right_on='new_card_id', how='left')

In [447]:
test = test.merge(hist2, left_on='card_id', right_on='new_card_id', how='left')

## Feature Selection

In [448]:
# le = LabelEncoder()
# le.fit(train.append(test).hist_purchase_year_month_mode.values)
# for df in [train, test]:
#     df['first_active'] = pd.to_datetime(df['first_active'])
#     df['temp'] = 1 - df.hist_category_1_label_mean\
#     df['hist_category_1_authorized_flag_Y_0_cnt'] = df.hist_category_1_authorized_flag_Y_size - df.hist_category_1_authorized_flag_Y_sum
#     df.hist_installments_authorized_flag_Y_cnt_std = np.around(df.hist_installments_authorized_flag_Y_cnt_std, 4)
#     df['temp'] = df.hist_month_lag_max - df.hist_month_lag_min
#     df.hist_purchase_year_month_mode = le.transform(df.hist_purchase_year_month_mode)
#     df['has_new_merchant'] = 1
#     df.loc[df.new_card_id_size == 0, 'has_new_merchant'] = 0
#     df['hist_purchase_year_max_first_active_year'] = (df.hist_purchase_year_month_day_max - df.first_active).dt.days
#     df['hist_purchase_year_min_first_active_year'] = (df.hist_purchase_year_month_day_min - df.first_active).dt.days

In [449]:
# train.to_csv('./data_feature_engineering/train_3-70479.csv', index=False)
# test.to_csv('./data_feature_engineering/test_3-70479.csv', index=False)

In [450]:
    'new_installments_authorized_flag_Y_mode', 'new_installments_authorized_flag_Y_size', 'new_installments_authorized_flag_Y_mean', 'new_installments_authorized_flag_Y_max', 'new_installments_authorized_flag_Y_var', 'new_installments_authorized_flag_Y_min', 'new_installments_authorized_flag_Y_null_cnt', 'new_installments_authorized_flag_Y_null_rate', 'new_installments_authorized_flag_Y_cnt_std'  


('new_installments_authorized_flag_Y_mode',
 'new_installments_authorized_flag_Y_size',
 'new_installments_authorized_flag_Y_mean',
 'new_installments_authorized_flag_Y_max',
 'new_installments_authorized_flag_Y_var',
 'new_installments_authorized_flag_Y_min',
 'new_installments_authorized_flag_Y_null_cnt',
 'new_installments_authorized_flag_Y_null_rate',
 'new_installments_authorized_flag_Y_cnt_std')

In [539]:
trainable_feature = [
    'feature_1', 'feature_2', 'feature_3', 
    'first_active_month', 'first_active_year', 'first_active_elapsed_time_from_trade', 'first_active_total_day',
    'hist_card_id_size', 'new_card_id_size',
    'hist_authorized_flag_label_mean', 'hist_authorized_flag_label_sum',
    'hist_city_id_mode_authorized_flag_category_1_Y', 'hist_city_id_nunique_authorized_flag_category_1_Y',
    'hist_category_1_authorized_flag_Y_mean',
    'hist_installments_authorized_flag_Y_max', 'hist_installments_authorized_flag_Y_min', 'hist_installments_authorized_flag_Y_null_cnt', 'hist_installments_authorized_flag_Y_var', 'hist_installments_authorized_flag_Y_cnt_std',
    'hist_merchant_category_id_authorized_flag_Y_cnt_std',
    'hist_merchant_id_mode', 'hist_merchant_id_nunique2', 'hist_merchant_id_null_cnt', 'hist_merchant_id_authorized_flag_Y_cnt_std',
    'hist_month_lag_min', 'hist_month_lag_max', 'hist_month_lag_mode',
    'hist_subsector_id_mode', 'hist_subsector_id_nunique',
    'hist_purchase_year_nunique', 'hist_purchase_year_max',
    'hist_purchase_month_nunique', 'hist_purchase_month_mode',
    'hist_purchase_hour_skew', 'hist_purchase_hour_var',
    'hist_purchase_weekofyear_max', 'hist_purchase_weekofyear_nunique',
    'hist_purchase_amount_trim_var',
    'hist_purchase_date_total_day_max',
    'hist_month_diff_from_last_trade_mode',
    'hist_purchase_year_month_day_duration', 'hist_purchase_year_max_first_active_year',
    'new_city_id_mode_authorized_flag_Y', 'new_city_id_nunique_authorized_flag_Y',
    'new_category_1_authorized_flag_Y_mean',
    'new_installments_mode', 'new_installments_sum', 'new_installments_mean', 'new_installments_var', 'new_installments_max', 'new_installments_min', 'new_installments_null_cnt',
    'new_month_lag_mode', 'new_month_lag_sum', 'new_month_lag_mean', 'new_month_lag_var', 'new_month_lag_max', 'new_month_lag_min', 'new_month_lag_skew',
    'new_purchase_amount_trim_sum', 'new_purchase_amount_trim_mean', 'new_purchase_amount_trim_var', 'new_purchase_amount_trim_max', 'new_purchase_amount_trim_min', 'new_purchase_amount_trim_skew',
    'hist_month_lag_median'
] + get_col('new_fromRefDate_sum', 'new_fromRefDate_quantileRange') + get_col('new_purchase_date_total_day_max', 'new_purchase_date_total_day_skew') + get_col('new_avg_sales_lag3_sum', 'new_city_ym_rate_quantileRange')

In [534]:
def get_col(start, end):
    result = []
    logit = 999
    for i, v in enumerate(train.columns.tolist()):
        if v == key:
            result.append(v)
            logit = i
        
        if logit < i:
            result.append(v)
        
        if v == end:
            break
    return result

In [535]:
train.head(3)

,card_id,feature_1,feature_2,feature_3,first_active_month,outliers,target,first_active,first_active_year,first_active_elapsed_time_from_trade,first_active_total_day,hist_authorized_flag_label_mode,hist_authorized_flag_label_sum,hist_authorized_flag_label_mean,hist_card_id_size,hist_city_id_mode,hist_city_id_nunique,hist_category_1_label_mode,hist_category_1_label_sum,hist_category_1_label_mean,hist_installments_mode,hist_installments_sum,hist_installments_mean,hist_installments_var,hist_installments_max,hist_installments_min,hist_installments_null_cnt,hist_category_3_label_mode,hist_category_3_label_mean,hist_merchant_category_id_mode,hist_merchant_category_id_nunique,hist_merchant_id_nunique,hist_month_lag_mode,hist_month_lag_sum,hist_month_lag_mean,hist_month_lag_var,hist_month_lag_max,hist_month_lag_min,hist_month_lag_skew,hist_purchase_date_max,hist_purchase_date_min,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_var,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_skew,hist_purchase_amount_over_550,hist_category_2_mode,hist_category_2_mean,hist_state_id_mode,hist_state_id_nunique,hist_subsector_id_mode,hist_subsector_id_nunique,hist_purchase_amount_trim_sum,hist_purchase_amount_trim_mean,hist_purchase_amount_trim_var,hist_purchase_amount_trim_max,hist_purchase_amount_trim_min,hist_purchase_amount_trim_skew,hist_purchase_year_mode,hist_purchase_year_nunique,hist_purchase_year_mean,hist_purchase_year_max,hist_purchase_year_min,hist_purchase_month_mode,hist_purchase_month_nunique,hist_purchase_month_mean,hist_purchase_month_max,hist_purchase_month_min,hist_purchase_day_mode,hist_purchase_day_nunique,hist_purchase_day_mean,hist_purchase_day_var,hist_purchase_day_max,hist_purchase_day_min,hist_purchase_day_skew,hist_purchase_hour_mode,hist_purchase_hour_nunique,hist_purchase_hour_mean,hist_purchase_hour_var,hist_purchase_hour_max,hist_purchase_hour_min,hist_purchase_hour_skew,hist_purchase_dayofweek_mode,hist_purchase_dayofweek_nunique,hist_purchase_dayofweek_mean,hist_purchase_dayofweek_max,hist_purchase_dayofweek_min,hist_purchase_weekofyear_mode,hist_purchase_weekofyear_nunique,hist_purchase_weekofyear_mean,hist_purchase_weekofyear_max,hist_purchase_weekofyear_min,hist_purchase_weekend_mode,hist_purchase_weekend_sum,hist_purchase_weekend_mean,hist_price_sum,hist_price_mean,hist_price_max,hist_price_min,hist_price_var,hist_month_diff_max,hist_month_diff_min,hist_month_diff_mean,hist_month_diff_var,hist_month_diff_skew,hist_Christmas_Day_2017_mean,hist_Mothers_Day_2017_mean,hist_fathers_day_2017_mean,hist_Children_day_2017_mean,hist_Valentine_Day_2017_mean,hist_Black_Friday_2017_mean,hist_Mothers_Day_2018_mean,hist_duration_mean,hist_duration_min,hist_duration_max,hist_duration_var,hist_duration_skew,hist_amount_month_ratio_mean,hist_amount_month_ratio_min,hist_amount_month_ratio_max,hist_amount_month_ratio_var,hist_amount_month_ratio_skew,hist_category_2_mean_mean,hist_category_2_min_mean,hist_category_2_max_mean,hist_category_2_sum_mean,hist_category_3_mean_mean,hist_category_3_min_mean,hist_category_3_max_mean,hist_category_3_sum_mean,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow,hist_purchase_date_uptomin,new_authorized_flag_mode,new_authorized_flag_sum,new_authorized_flag_mean,new_card_id_size,new_city_id_mode,new_city_id_nunique,new_category_1_mode,new_category_1_sum,new_category_1_mean,new_installments_mode,new_installments_sum,new_installments_mean,new_installments_var,new_installments_max,new_installments_min,new_installments_null_cnt,new_category_3_mode,new_category_3_mean,new_merchant_category_id_mode,new_merchant_category_id_nunique,new_merchant_id_nunique,new_month_lag_mode,new_month_lag_sum,new_month_lag_mean,new_month_lag_var,new_month_lag_max,new_month_lag_min,new_month_lag_skew,new_purchase_date_max,new_purchase_date_min,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_var,new_purchase_amount_max,new_purchase_am

In [540]:
    'new_city_id_mode', 'new_city_id_nunique',
    'new_category_1_mode', 'new_category_1_sum', 'new_category_1_mean',
    'new_installments_mode', 'new_installments_sum', 'new_installments_mean', 'new_installments_var', 'new_installments_max', 'new_installments_min', 'new_installments_null_cnt',
    'new_month_lag_mode', 'new_month_lag_sum', 'new_month_lag_mean', 'new_month_lag_var', 'new_month_lag_max', 'new_month_lag_min', 'new_month_lag_skew',
    'new_purchase_amount_trim_sum', 'new_purchase_amount_trim_mean', 'new_purchase_amount_trim_var', 'new_purchase_amount_trim_max', 'new_purchase_amount_trim_min', 'new_purchase_amount_trim_skew',
    'new_month_diff_max', 'new_month_diff_min', 'new_month_diff_mean', 'new_month_diff_var', 'new_month_diff_skew',
    'card_id_total_size', 'card_id_size_ratio', 'purchase_amount_total', 'purchase_amount_mean', 'purchase_amount_max', 'purchase_amount_min', 'purchase_amount_ratio',
    'month_diff_mean', 'month_diff_ratio', 'month_lag_mean', 'month_lag_max', 'month_lag_min',
    'category_1_mean', 'installments_total', 'installments_mean', 'installments_max', 'installments_ratio',
    'price_total', 'price_mean', 'price_max', 'duration_mean', 'duration_min', 'duration_max',
    'amount_month_ratio_mean', 'amount_month_ratio_min', 'amount_month_ratio_max',
    'new_CLV', 'hist_CLV', 'CLV_ratio'

('new_CLV', 'hist_CLV', 'CLV_ratio')

In [541]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 1,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 8,
         "random_state": 4590}

In [542]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=15)

# train_columns = [f for f in train.columns if f not in FEATS_EXCLUDED]
train_columns = trainable_feature

oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros(len(test))
feature_importance = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train['outliers'].values)):    
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][train_columns], label=train.iloc[trn_idx]['target'])
    val_data = lgb.Dataset(train.iloc[val_idx][train_columns], label=train.iloc[val_idx]['target'])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
    oof_lgb[val_idx] = clf.predict(train.iloc[val_idx][train_columns], num_iteration=clf.best_iteration)
    
    predictions_lgb += clf.predict(test[train_columns], num_iteration=clf.best_iteration) / folds.n_splits
    
    fold_importance = pd.DataFrame()
    fold_importance["Feature"] = train_columns
    fold_importance["importance"] = clf.feature_importance()
    fold_importance["fold"] = fold_ + 1
    feature_importance = pd.concat([feature_importance, fold_importance], axis=0)
    
print("CV score: {:<8.5f}".format(mean_squared_error(train.target.values, oof_lgb)**0.5))

fold n°0
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.65943	valid_1's rmse: 3.71304
[200]	training's rmse: 3.57708	valid_1's rmse: 3.68496
[300]	training's rmse: 3.52599	valid_1's rmse: 3.67374


KeyboardInterrupt: 

score : 3.70479

In [479]:
sub = pd.read_csv('./data/sample_submission.csv')

In [480]:
sub.target = predictions_lgb

In [481]:
sub.to_csv('./data/sub_3-65066.csv', index=False)